- In this Notebook, I am trying to integrate the training data from the backscore and rest of the period from prj-prod-dataplatform.audit_balance.ml_model_run_details table.
- In this I will compare the training period with each month of test period.

# **PSI - CSI Calculation**

---







## Define Libraries

In [2]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
import tempfile
import time
from datetime import datetime
import uuid
import joblib
import uuid

import gcsfs
import duckdb as dd
import pickle
import joblib
from typing import Union
import io
path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)

### Function

#### expand_calc_features

In [3]:
import pandas as pd
import json

def expand_calc_features(df):
    """
    Expand the calcFeatures JSON column into separate columns and return the complete DataFrame.

    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data

    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """

    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()

    # Parse the calcFeatures JSON column
    calc_features_list = []

    for idx, calc_features_str in enumerate(df['calcFeatures']):
        try:
            # Parse the JSON string
            features_dict = json.loads(calc_features_str.replace("'", '"'))  # Replace single quotes with double quotes for valid JSON
            calc_features_list.append(features_dict)
        except (json.JSONDecodeError, AttributeError) as e:
            # If parsing fails, create an empty dict and print warning
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            calc_features_list.append({})

    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_list)

    # Add prefix to JSON-derived columns to avoid conflicts
    calc_features_df = calc_features_df.add_prefix('calc_')

    # Reset index to ensure proper alignment
    df_expanded = df_expanded.reset_index(drop=True)
    calc_features_df = calc_features_df.reset_index(drop=True)

    # Combine original DataFrame with expanded calcFeatures
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)

    return result_df


#### expand_calc_features_robust

In [4]:
import pandas as pd
import json

def expand_calc_features_robust(df):
    """
    Expand the calcFeatures JSON column into separate columns with better error handling.

    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data

    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """

    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()

    # Parse the calcFeatures JSON column
    calc_features_data = []

    for idx, row in df.iterrows():
        calc_features_str = row['calcFeatures']

        if pd.isna(calc_features_str) or calc_features_str == '':
            calc_features_data.append({})
            continue

        try:
            # Clean the string and parse JSON
            cleaned_str = calc_features_str.replace("'", '"').replace('None', 'null').replace('True', 'true').replace('False', 'false')
            features_dict = json.loads(cleaned_str)
            calc_features_data.append(features_dict)
        except Exception as e:
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            print(f"Problematic string: {calc_features_str[:100]}...")  # Print first 100 chars
            calc_features_data.append({})

    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_data)

    # Add prefix to JSON-derived columns to avoid conflicts with existing columns
    calc_features_df = calc_features_df.add_prefix('feat_')

    # Combine DataFrames
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)

    print(f"Original DataFrame shape: {df.shape}")
    print(f"Expanded DataFrame shape: {result_df.shape}")
    print(f"Added {len(calc_features_df.columns)} new columns from calcFeatures")

    return result_df

#### transform_data

In [15]:
import pandas as pd
import json
import uuid
from datetime import datetime
from typing import List

def transform_data(d1: pd.DataFrame, feature_column: List[str], a='demo_score', modelDisplayName = 'Cash_beta_trench1_Demo_backscore', subscription_name = 'sil_march 25 models'):
    # Read the input CSV file
    df = d1.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = feature_column
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get(a, 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":modelDisplayName,
            "modelVersionId":"v1",
            "subscription_name": subscription_name,
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

#### PSI Functions new

In [5]:
## Updated on 27-10-2025 - Modified for Training Period Baseline
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

def identify_feature_types(df: pd.DataFrame, feature_list: List[str]) -> Dict[str, List[str]]:
    """
    Identify categorical and numerical features from the feature list.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to classify

    Returns:
    --------
    Dict with 'categorical' and 'numerical' keys containing respective feature lists
    """
    categorical_features = []
    numerical_features = []

    for feature in feature_list:
        if feature not in df.columns:
            print(f"Warning: Feature '{feature}' not found in dataframe")
            continue

        # Check if feature is numeric
        if pd.api.types.is_numeric_dtype(df[feature]):
            # If unique values are less than 15 and all integers, treat as categorical
            unique_vals = df[feature].nunique()
            if unique_vals < 15 and df[feature].dropna().apply(lambda x: x == int(x) if isinstance(x, (int, float)) else False).all():
                categorical_features.append(feature)
            else:
                numerical_features.append(feature)
        else:
            categorical_features.append(feature)

    return {
        'categorical': categorical_features,
        'numerical': numerical_features
    }


def create_bins_for_features(df: pd.DataFrame,
                             numerical_features: List[str],
                             categorical_features: List[str],
                             train_period_df: pd.DataFrame) -> Dict:
    """
    Create bins for numerical features (deciles with fallback) and categorical features (top 6 + others)
    based on the entire training period data.

    Parameters:
    -----------
    df : pd.DataFrame
        Full input dataframe
    numerical_features : List[str]
        List of numerical features
    categorical_features : List[str]
        List of categorical features
    train_period_df : pd.DataFrame
        Training period dataframe (June 2024 to March 2025)

    Returns:
    --------
    Dictionary containing binning information for each feature
    """
    binning_info = {}

    # Create bins for numerical features with fallback strategy
    for feature in numerical_features:
        valid_data = train_period_df[feature].dropna()

        if len(valid_data) == 0:
            binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
            continue

        bins = None
        bin_count = None

        # Try 10 bins (deciles)
        try:
            test_bins = np.percentile(valid_data, np.arange(0, 101, 10))
            test_bins = np.unique(test_bins)
            if len(test_bins) >= 11:  # 11 edges = 10 bins
                bins = test_bins
                bin_count = 10
        except Exception as e:
            pass

        # If 10 bins not possible, try 5 bins
        if bins is None:
            try:
                test_bins = np.percentile(valid_data, np.arange(0, 101, 20))
                test_bins = np.unique(test_bins)
                if len(test_bins) >= 6:  # 6 edges = 5 bins
                    bins = test_bins
                    bin_count = 5
            except Exception as e:
                pass

        # If 5 bins not possible, try 3 bins
        if bins is None:
            try:
                test_bins = np.percentile(valid_data, [0, 33.33, 66.67, 100])
                test_bins = np.unique(test_bins)
                if len(test_bins) >= 4:  # 4 edges = 3 bins
                    bins = test_bins
                    bin_count = 3
            except Exception as e:
                pass

        # If still no bins possible, use equal distance bins of 5
        if bins is None:
            print(f"Warning: Feature '{feature}' has insufficient variance - cannot create standard bins")
            print(f"Feature '{feature}': Using equal distance bins of 5")

            min_val = valid_data.min()
            max_val = valid_data.max()

            # Create 5 equal distance bins
            bins = np.linspace(min_val, max_val, 6)  # 6 edges = 5 bins
            bins = np.unique(bins)
            bin_count = len(bins) - 1

            # If all values are the same, add slight buffer
            if bin_count == 1:
                bins = np.array([min_val - 0.1, min_val, min_val + 0.1])
                bin_count = 2
                print(f"Feature '{feature}': Constant value ({min_val}). Created 2 equal distance bins with buffer")

        # Add infinity edges to capture all values
        bins = bins.copy()
        bins[0] = -np.inf
        bins[-1] = np.inf

        print(f"Feature '{feature}': Created {bin_count} bins")

        # Create bin ranges dictionary
        bin_ranges = {}
        for i in range(len(bins)-1):
            bin_name = f"Bin_{i+1}"
            bin_ranges[bin_name] = {
                'min': bins[i],
                'max': bins[i+1],
                'range_str': f"[{bins[i]:.2f}, {bins[i+1]:.2f}]" if not np.isinf(bins[i]) and not np.isinf(bins[i+1]) else f"({bins[i]}, {bins[i+1]})"
            }

        binning_info[feature] = {
            'type': 'numerical',
            'bins': bins,
            'bin_ranges': bin_ranges,
            'bin_count': bin_count
        }

    # Create bins for categorical features (top 6 + others) using training period
    for feature in categorical_features:
        value_counts = train_period_df[feature].value_counts()
        unique_categories = value_counts.index.tolist()
        print(f"Unique categories: {unique_categories}")

        if len(unique_categories) <= 6:
            # Treat each category as a separate bin
            top_categories = unique_categories
        else:
            # Use top 6 categories only
            top_categories = value_counts.nlargest(6).index.tolist()

        print(f"Top categories for feature '{feature}': {top_categories}")

        binning_info[feature] = {
                'type': 'categorical',
                'top_categories': top_categories,
                'bin_ranges': {}  # No ranges for categorical
            }

    return binning_info


def apply_binning(df: pd.DataFrame,
                  feature: str,
                  binning_info: Dict) -> pd.Series:
    """
    Apply binning to a feature based on binning information.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature : str
        Feature name
    binning_info : Dict
        Binning information for the feature

    Returns:
    --------
    pd.Series with binned values
    """
    if binning_info['type'] == 'numerical':
        if binning_info['bins'] is None:
            return pd.Series(['Missing'] * len(df), index=df.index)

        bins = binning_info['bins']
        labels = [f"Bin_{i+1}" for i in range(len(bins)-1)]

        binned = pd.cut(df[feature],
                       bins=bins,
                       labels=labels,
                       include_lowest=True,
                       duplicates='drop')

        # Handle nulls - convert to string and then replace
        binned = binned.astype(str)
        binned[df[feature].isna()] = 'Missing'

        return binned

    else:  # categorical
        top_cats = binning_info['top_categories']

        # Convert to string for consistent comparison
        if pd.api.types.is_categorical_dtype(df[feature]):
            feature_data = df[feature].astype(str)
        else:
            feature_data = df[feature].astype(str)

        # Replace NaN string representation with 'Missing'
        feature_data = feature_data.replace('nan', 'Missing')

        # Convert top_cats to strings for comparison
        top_cats_str = [str(cat) for cat in top_cats]

        # Apply binning logic: use category name if in top_cats, else 'Others' (except for Missing)
        binned = feature_data.apply(lambda x: x if x in top_cats_str else ('Others' if x != 'Missing' else 'Missing'))

        return binned


def calculate_psi(expected_pct: pd.Series,
                  actual_pct: pd.Series,
                  epsilon: float = 0.0001) -> float:
    """
    Calculate Population Stability Index with proper epsilon handling and renormalization.

    Parameters:
    -----------
    expected_pct : pd.Series
        Expected (baseline) percentages
    actual_pct : pd.Series
        Actual percentages
    epsilon : float
        Small value to avoid log(0)

    Returns:
    --------
    PSI value
    """
    # Align indices
    all_bins = expected_pct.index.union(actual_pct.index)
    expected_pct = expected_pct.reindex(all_bins, fill_value=0)
    actual_pct = actual_pct.reindex(all_bins, fill_value=0)

    # Only add epsilon where values are zero
    expected_pct = expected_pct.apply(lambda x: epsilon if x == 0 else x)
    actual_pct = actual_pct.apply(lambda x: epsilon if x == 0 else x)

    # Renormalize to ensure they sum to 1 after adding epsilon
    expected_pct = expected_pct / expected_pct.sum()
    actual_pct = actual_pct / actual_pct.sum()

    # Calculate PSI
    psi_value = np.sum((actual_pct - expected_pct) * np.log(actual_pct / expected_pct))

    return psi_value


def calculate_month_on_month_psi(df: pd.DataFrame,
                                 feature_list: List[str],
                                 segment_columns: List[str],
                                 month_col: str = 'Application_month',
                                 data_selection_col: str = 'Data_selection',
                                 account_id_col: str = 'digitalLoanAccountId') -> pd.DataFrame:
    """
    Calculate PSI for each feature comparing training period (June 2024 to March 2025)
    vs each month after March 2025, overall and by segments.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to calculate PSI for
    segment_columns : List[str]
        List of segment columns
    month_col : str
        Name of month column
    data_selection_col : str
        Name of data selection column (identifies train period)
    account_id_col : str
        Name of account ID column for counting distinct accounts

    Returns:
    --------
    pd.DataFrame with PSI values with one row per feature-month-segment combination
    """
    # Create a copy to avoid modifying original
    df = df.copy()

    # Identify training and test periods
    train_df = df[df[data_selection_col] == 'Train'].copy()
    test_df = df[df[data_selection_col] != 'Train'].copy()

    if len(train_df) == 0:
        raise ValueError("No training data found. Check Data_selection column.")

    print(f"Training period: {train_df[month_col].min()} to {train_df[month_col].max()}")
    print(f"Test period: {test_df[month_col].min()} to {test_df[month_col].max()}")

    # Identify feature types
    feature_types = identify_feature_types(df, feature_list)

    # Create binning strategy based on training period
    binning_info = create_bins_for_features(
        df,
        feature_types['numerical'],
        feature_types['categorical'],
        train_df
    )

    # Get sorted test months
    test_months = sorted(test_df[month_col].unique())

    results = []

    # Calculate overall PSI
    for feature in feature_list:
        if feature not in df.columns:
            continue

        # Apply binning to entire dataset
        df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        # print(f"Feature binned {df[f'{feature}_binned']}")
        # Get training period distribution (baseline)
        train_baseline = df[df[data_selection_col] == 'Train'][f'{feature}_binned'].value_counts(normalize=True)

        # Calculate PSI for each test month
        for month in test_months:
            actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
            psi_value = calculate_psi(train_baseline, actual_dist)

            # Calculate average percentages across all bins
            expected_avg_pct = train_baseline.mean() * 100
            actual_avg_pct = actual_dist.mean() * 100

            # # Count distinct accounts for segment
            # base_segment_count = train_segment[account_id_col].nunique()
            # actual_segment_count = actual_segment[account_id_col].nunique()


            results.append({
                'Feature': feature,
                'Feature_Type': binning_info[feature]['type'],
                'Segment_Column': 'Overall',
                'Segment_Value': 'All',
                'Month': f"{month}",
                'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                'Current_Month': month,
                'Expected_Percentage': expected_avg_pct,
                'Actual_Percentage': actual_avg_pct,
                'PSI': psi_value
            })

    # Calculate PSI by segments
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            continue

        segments = df[segment_col].dropna().unique()

        for segment_val in segments:
            segment_df = df[df[segment_col] == segment_val]

            for feature in feature_list:
                if feature not in df.columns:
                    continue

                # Get training period distribution for segment
                train_segment = segment_df[segment_df[data_selection_col] == 'Train']
                if len(train_segment) == 0:
                    continue

                train_baseline = train_segment[f'{feature}_binned'].value_counts(normalize=True)

                # Calculate PSI for each test month
                for month in test_months:
                    actual_segment = segment_df[segment_df[month_col] == month]
                    if len(actual_segment) == 0:
                        continue

                    actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
                    psi_value = calculate_psi(train_baseline, actual_dist)

                    # Calculate average percentages across all bins
                    expected_avg_pct = train_baseline.mean() * 100
                    actual_avg_pct = actual_dist.mean() * 100

                    # Count distinct accounts for segment
                    base_segment_count = train_segment[account_id_col].nunique()
                    actual_segment_count = actual_segment[account_id_col].nunique()

                    results.append({
                        'Feature': feature,
                        'Feature_Type': binning_info[feature]['type'],
                        'Segment_Column': segment_col,
                        'Segment_Value': segment_val,
                        'Month': f"{month}",
                        'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                        'Current_Month': month,
                        'Base_Count': base_segment_count,
                        'Actual_Count': actual_segment_count,
                        'Expected_Percentage': expected_avg_pct,
                        'Actual_Percentage': actual_avg_pct,
                        'PSI': psi_value
                    })

    return pd.DataFrame(results)


def calculate_bin_level_psi(df: pd.DataFrame,
                            feature_list: List[str],
                            segment_columns: List[str],
                            month_col: str = 'Application_month',
                            data_selection_col: str = 'Data_selection',
                            account_id_col: str = 'digitalLoanAccountId') -> pd.DataFrame:
    """
    Calculate bin-level PSI for each feature comparing training period
    vs each month after March 2025, overall and by segments.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to calculate PSI for
    segment_columns : List[str]
        List of segment columns
    month_col : str
        Name of month column
    data_selection_col : str
        Name of data selection column
    account_id_col : str
        Name of account ID column for counting distinct accounts

    Returns:
    --------
    pd.DataFrame with bin-level PSI details including bin ranges
    """
    # Create a copy to avoid modifying original
    df = df.copy()

    # Identify training and test periods
    train_df = df[df[data_selection_col] == 'Train'].copy()
    test_df = df[df[data_selection_col] != 'Train'].copy()

    if len(train_df) == 0:
        raise ValueError("No training data found. Check Data_selection column.")

    print(f"Training period: {train_df[month_col].min()} to {train_df[month_col].max()}")
    print(f"Test period: {test_df[month_col].min()} to {test_df[month_col].max()}")

    # Identify feature types
    feature_types = identify_feature_types(df, feature_list)

    # Create binning strategy based on training period
    binning_info = create_bins_for_features(
        df,
        feature_types['numerical'],
        feature_types['categorical'],
        train_df
    )

    # Get sorted test months
    test_months = sorted(test_df[month_col].unique())

    results = []
    epsilon = 0.0001

    # Calculate overall bin-level PSI
    for feature in feature_list:
        if feature not in df.columns:
            continue

        # Apply binning to entire dataset
        df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        # print(df[f'{feature}_binned'])

        # Get training period distribution (baseline)
        train_baseline = df[df[data_selection_col] == 'Train'][f'{feature}_binned'].value_counts(normalize=True)

        # Calculate bin-level PSI for each test month
        for month in test_months:
            month_data = df[df[month_col] == month]
            actual_dist = month_data[f'{feature}_binned'].value_counts(normalize=True)

            # Count distinct accounts
            base_count = df[df[data_selection_col] == 'Train'][account_id_col].nunique()
            actual_count = month_data[account_id_col].nunique()

            # Get all bins
            all_bins = train_baseline.index.union(actual_dist.index)

            for bin_name in all_bins:
                # Simplified epsilon logic - no redundancy
                expected_pct = train_baseline.get(bin_name, 0)
                actual_pct = actual_dist.get(bin_name, 0)

                # Add epsilon only if zero
                expected_pct = epsilon if expected_pct == 0 else expected_pct
                actual_pct = epsilon if actual_pct == 0 else actual_pct

                # Calculate bin-level PSI
                bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct)

                # Get bin range information
                bin_ranges = binning_info[feature]['bin_ranges']
                if bin_name in bin_ranges:
                    bin_min = bin_ranges[bin_name]['min']
                    bin_max = bin_ranges[bin_name]['max']
                    bin_range = bin_ranges[bin_name]['range_str']
                else:
                    # For categorical or special bins (Missing, Others)
                    bin_min = None
                    bin_max = None
                    bin_range = bin_name

                results.append({
                    'Feature': feature,
                    'Feature_Type': binning_info[feature]['type'],
                    'Segment_Column': 'Overall',
                    'Segment_Value': 'All',
                    'Month': f"{month}",
                    'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                    'Current_Month': month,
                    'Base_Count': base_count,
                    'Actual_Count': actual_count,
                    'Bin': bin_name,
                    'Bin_Range': bin_range,
                    'Bin_Min': bin_min,
                    'Bin_Max': bin_max,
                    'Base_Percentage': (train_baseline.get(bin_name, 0) * 100),
                    'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
                    'Bin_PSI': bin_psi
                })

    # Calculate bin-level PSI by segments
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            continue

        segments = df[segment_col].dropna().unique()

        for segment_val in segments:
            segment_df = df[df[segment_col] == segment_val]

            for feature in feature_list:
                if feature not in df.columns:
                    continue

                # Get training period distribution for segment
                train_segment = segment_df[segment_df[data_selection_col] == 'Train']
                if len(train_segment) == 0:
                    continue

                train_baseline = train_segment[f'{feature}_binned'].value_counts(normalize=True)

                # Calculate bin-level PSI for each test month
                for month in test_months:
                    actual_segment = segment_df[segment_df[month_col] == month]
                    if len(actual_segment) == 0:
                        continue

                    actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)

                    # Count distinct accounts for segment
                    base_segment_count = train_segment[account_id_col].nunique()
                    actual_segment_count = actual_segment[account_id_col].nunique()

                    # Get all bins
                    all_bins = train_baseline.index.union(actual_dist.index)

                    for bin_name in all_bins:
                        # Simplified epsilon logic - no redundancy
                        expected_pct = train_baseline.get(bin_name, 0)
                        actual_pct = actual_dist.get(bin_name, 0)

                        # Add epsilon only if zero
                        expected_pct = epsilon if expected_pct == 0 else expected_pct
                        actual_pct = epsilon if actual_pct == 0 else actual_pct

                        # Calculate bin-level PSI
                        bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct)

                        # Get bin range information
                        bin_ranges = binning_info[feature]['bin_ranges']
                        if bin_name in bin_ranges:
                            bin_min = bin_ranges[bin_name]['min']
                            bin_max = bin_ranges[bin_name]['max']
                            bin_range = bin_ranges[bin_name]['range_str']
                        else:
                            # For categorical or special bins (Missing, Others)
                            bin_min = None
                            bin_max = None
                            bin_range = bin_name

                        results.append({
                            'Feature': feature,
                            'Feature_Type': binning_info[feature]['type'],
                            'Segment_Column': segment_col,
                            'Segment_Value': segment_val,
                            'Month': f"{month}",
                            'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                            'Current_Month': month,
                            'Base_Count': base_segment_count,
                            'Actual_Count': actual_segment_count,
                            'Bin': bin_name,
                            'Bin_Range': bin_range,
                            'Bin_Min': bin_min,
                            'Bin_Max': bin_max,
                            'Base_Percentage': (train_baseline.get(bin_name, 0) * 100),
                            'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
                            'Bin_PSI': bin_psi
                        })

    return pd.DataFrame(results)

## Queries

### SIL

#### Alpha - CIC-SIL-Model

##### Query from ml_model_run_details

In [8]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName = 'Alpha - CIC-SIL-Model')
SELECT
  r.customerId,r.digitalLoanAccountId,prediction Alpha_cic_sil_score
    ,start_time,end_time,modelDisplayName,modelVersionId,
   loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
        when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
        when lower(loanmaster.deviceType) like '%andro%' then 'android'
        else 'ios' end osType,
 'Alpha - CIC-SIL-Model' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature calcFeatures,
  'Test' Data_selection,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
;
"""

dfd = client.query(sq).to_dataframe()
print(f"The shape of the dataframe is: {dfd.shape}")
dfd.head()

The shape of the dataframe is: (70128, 19)


,customerId,digitalLoanAccountId,Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month
0,1631839,7bc444a0-c9b5-4209-ade6-5093c03e1b3c,0.16220137019747308,2025-07-06 09:44:43.304546,2025-07-06 09:44:43.310956,Alpha - CIC-SIL-Model,v1,SIL Competitor,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1751760000000,""cic_Personal_Loans_...",Test,2025-07-06 17:44:34,NaT,2025-07
1,1700061,f1d58bdf-864c-4823-9c49-b06f137b56a3,0.212585800166342,2025-07-24 06:50:38.752444,2025-07-24 06:50:38.759082,Alpha - CIC-SIL-Model,v1,SIL Competitor,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1753315200000,""cic_Personal_Loans_...",Test,2025-07-24 14:50:27,NaT,2025-07
2,1797942,707de144-2c87-47bc-b9cf-172c7fa72d35,0.0847815387303776,2025-10-03 05:55:52.545704,2025-10-03 05:55:52.552438,Alpha - CIC-SIL-Model,v1,SIL Competitor,F,Appliance,ios,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1759449600000,""cic_Personal_Loans_...",Test,2025-10-03 13:55:42,2025-10-03 14:08:22,2025-10
3,2156466,6d364edf-1c66-4487-94fb-6392937aa8a7,0.123341079284235,2025-10-17 03:08:44.796166,2025-10-17 03:08:44.802866,Alpha - CIC-SIL-Model,v1,SIL-Instore,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1760659200000,""cic_Personal_Loans_...",Test,2025-10-17 11:08:33,NaT,2025-10
4,2159532,51266e7d-3f13-431a-8207-b7e27f3940a8,0.19535727112153314,2025-08-29 11:03:36.813002,2025-08-29 11:03:36.819676,Alpha - CIC-SIL-Model,v1,SIL-Instore,M,Mall,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1756425600000,""cic_Personal_Loans_...",Test,2025-08-29 19:03:27,2025-09-01 14:19:13,2025-08


###### expanded_df Part 1

In [9]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 19
Expanded columns: 29


,customerId,digitalLoanAccountId,Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month,calc_run_date,calc_cic_Personal_Loans_granted_contracts_amt_24M,calc_cic_days_since_last_inquiry,calc_cic_cnt_active_contracts,calc_cic_vel_contract_nongranted_cnt_12on24,calc_cic_max_amt_granted_24M,calc_cic_zero_non_granted_ever_flag,calc_cic_tot_active_contracts_util,calc_cic_vel_contract_granted_amt_12on24,calc_cic_zero_granted_ever_flag
0,1631839,7bc444a0-c9b5-4209-ade6-5093c03e1b3c,0.16220137019747308,2025-07-06 09:44:43.304546,2025-07-06 09:44:43.310956,Alpha - CIC-SIL-Model,v1,SIL Competitor,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1751760000000,""cic_Personal_Loans_...",Test,2025-07-06 17:44:34,NaT,2025-07,1.751760e+12,NaN,453.0,NaN,NaN,NaN,0,NaN,NaN,0
1,1700061,f1d58bdf-864c-4823-9c49-b06f137b56a3,0.212585800166342,2025-07-24 06:50:38.752444,2025-07-24 06:50:38.759082,Alpha - CIC-SIL-Model,v1,SIL Competitor,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1753315200000,""cic_Personal_Loans_...",Test,2025-07-24 14:50:27,NaT,2025-07,1.753315e+12,NaN,0.0,1.0,1.994012,NaN,0,0.000000,NaN,0
2,1797942,707de144-2c87-47bc-b9cf-172c7fa72d35,0.0847815387303776,2025-10-03 05:55:52.545704,2025-10-03 05:55:52.552438,Alpha - CIC-SIL-Model,v1,SIL Competitor,F,Appliance,ios,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1759449600000,""cic_Personal_Loans_...",Test,2025-10-03 13:55:42,2025-10-03 14:08:22,2025-10,1.759450e+12,119845.0,67.0,23.0,1.336000,18206.0,0,0.616305,0.774057,0
3,2156466,6d364edf-1c66-4487-94fb-6392937aa8a7,0.123341079284235,2025-10-17 03:08:44.796166,2025-10-17 03:08:44.802866,Alpha - CIC-SIL-Model,v1,SIL-Instore,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1760659200000,""cic_Personal_Loans_...",Test,2025-10-17 11:08:33,NaT,2025-10,1.760659e+12,NaN,896.0,NaN,NaN,NaN,1,NaN,NaN,0
4,2159532,51266e7d-3f13-431a-8207-b7e27f3940a8,0.19535727112153314,2025-08-29 11:03:36.813002,2025-08-29 11:03:36.819676,Alpha - CIC-SIL-Model,v1,SIL-Instore,M,Mall,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1756425600000,""cic_Personal_Loans_...",Test,2025-08-29 19:03:27,2025-09-01 14:19:13,2025-08,1.756426e+12,NaN,273.0,NaN,2.012048,NaN,0,NaN,NaN,0


In [10]:
expanded_df.groupby('Application_month').size()

Application_month
2025-03      180
2025-04      541
2025-05      858
2025-06     9934
2025-07    14043
2025-08    15085
2025-09    13880
2025-10    15607
dtype: int64

##### Query from risk_mart.sil_risk_ds_master_20230101_20250309_v2

In [18]:
sq = """select 
    customerId customer_id ,
    digitalLoanAccountId,
    cic_score,
    cic_Personal_Loans_granted_contracts_amt_24M,
    cic_days_since_last_inquiry, 
    cic_cnt_active_contracts,
    cic_vel_contract_nongranted_cnt_12on24,
    cic_max_amt_granted_24M, 
    cic_zero_non_granted_ever_flag,
    cic_tot_active_contracts_util,
    cic_vel_contract_granted_amt_12on24,
    cic_zero_granted_ever_flag
from risk_mart.sil_risk_ds_master_20230101_20250309_v2;"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID d3f12f89-cb4b-4e08-b3bb-d87a3dc4cb14 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (349396, 12)


In [19]:
data.head()

,customer_id,digitalLoanAccountId,cic_score,cic_Personal_Loans_granted_contracts_amt_24M,cic_days_since_last_inquiry,cic_cnt_active_contracts,cic_vel_contract_nongranted_cnt_12on24,cic_max_amt_granted_24M,cic_zero_non_granted_ever_flag,cic_tot_active_contracts_util,cic_vel_contract_granted_amt_12on24,cic_zero_granted_ever_flag
0,2573242,aa7593fd-f43e-4680-8765-08ca9b0a2c9c,0.086571,50746.0,131.0,NaN,NaN,18006.0,1,NaN,1.058382,0
1,2814303,1a2cb6f2-e66f-444a-ba22-a05cb5652ef2,0.175819,NaN,0.0,NaN,2.012048,45000.0,0,NaN,NaN,0
2,2960153,89b555bd-cdb6-4724-ab4b-5c8eee7ad61e,0.125669,NaN,1284.0,NaN,NaN,NaN,1,NaN,NaN,0
3,2664187,a731f941-ed8a-4278-9ba4-1fd41132aa5e,0.117005,NaN,267.0,NaN,NaN,10000.0,1,NaN,1.000000,0
4,2494260,9b118965-74ea-4832-a640-8b5ea2cc140e,0.125541,NaN,1531.0,NaN,NaN,NaN,1,NaN,NaN,0


In [21]:
feature_column = ['cic_Personal_Loans_granted_contracts_amt_24M',
       'cic_days_since_last_inquiry', 'cic_cnt_active_contracts',
       'cic_vel_contract_nongranted_cnt_12on24', 'cic_max_amt_granted_24M',
       'cic_zero_non_granted_ever_flag', 'cic_tot_active_contracts_util',
       'cic_vel_contract_granted_amt_12on24', 'cic_zero_granted_ever_flag']

In [22]:
dfd = transform_data(data, feature_column, a='cic_score', modelDisplayName='Alpha - CIC-SIL-Model') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,2573242,aa7593fd-f43e-4680-8765-08ca9b0a2c9c,1a3d2c91-0d0b-4332-948c-78b84433da69,0.086571,2025-10-30T17:24:47.238086,2025-10-30T17:24:47.238086,Alpha - CIC-SIL-Model,v1,sil_march 25 models,3c2f50ac-6344-49d1-ab67-50df03257a2e,2025-10-30T17:24:47.238086,{},"{""cic_Personal_Loans_granted_contracts_amt_24M..."
1,2814303,1a2cb6f2-e66f-444a-ba22-a05cb5652ef2,79ba24f0-a005-4bcc-a1a0-1ffe858ac272,0.175819,2025-10-30T17:24:47.238086,2025-10-30T17:24:47.238086,Alpha - CIC-SIL-Model,v1,sil_march 25 models,1d45b0ae-295f-4300-83dc-b73b0a036fd0,2025-10-30T17:24:47.238086,{},"{""cic_days_since_last_inquiry"": 0.0, ""cic_vel_..."
2,2960153,89b555bd-cdb6-4724-ab4b-5c8eee7ad61e,366123dd-8c4a-430d-8adc-58bc9b98c79b,0.125669,2025-10-30T17:24:47.238086,2025-10-30T17:24:47.238086,Alpha - CIC-SIL-Model,v1,sil_march 25 models,afcb54c6-dc62-4783-b779-f0d00daf2c34,2025-10-30T17:24:47.238086,{},"{""cic_days_since_last_inquiry"": 1284.0, ""cic_z..."
3,2664187,a731f941-ed8a-4278-9ba4-1fd41132aa5e,87b75eb5-3cdf-4d03-8627-4b59d20d624d,0.117005,2025-10-30T17:24:47.239087,2025-10-30T17:24:47.239087,Alpha - CIC-SIL-Model,v1,sil_march 25 models,b326c355-69b1-4e0a-99a5-e9502ada5c2f,2025-10-30T17:24:47.239087,{},"{""cic_days_since_last_inquiry"": 267.0, ""cic_ma..."
4,2494260,9b118965-74ea-4832-a640-8b5ea2cc140e,d1cc7eaa-f6b8-4f5c-9f41-0d6ac1c23b0f,0.125541,2025-10-30T17:24:47.239087,2025-10-30T17:24:47.239087,Alpha - CIC-SIL-Model,v1,sil_march 25 models,45d6426d-1ead-4201-ae27-cbb2f24dad64,2025-10-30T17:24:47.239087,{},"{""cic_days_since_last_inquiry"": 1531.0, ""cic_z..."


In [23]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=c7c5ef53-3cee-46e5-b93b-0b67f1843429>

In [29]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details`
  WHERE modelDisplayName = 'Alpha - CIC-SIL-Model')
SELECT
  r.customerId,r.digitalLoanAccountId,prediction Alpha_cic_sil_score
    ,start_time,end_time,modelDisplayName,modelVersionId,
   loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
        when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
        when lower(loanmaster.deviceType) like '%andro%' then 'android'
        else 'ios' end osType,
 'Alpha - CIC-SIL-Model' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature calcFeatures,
  'Train' Data_selection,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  date(r.start_time)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
;
"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is: {dfd.shape}")
dfd.head()

The shape of the dataframe is: (349207, 19)


,customerId,digitalLoanAccountId,Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month
0,1125881,aac8ee2c-dfcc-4b20-bece-61a48deb1723,0.109893,2025-10-30T17:24:52.984291,2025-10-30T17:24:52.984291,Alpha - CIC-SIL-Model,v1,SIL-Instore,F,Appliance,ios,Alpha - CIC-SIL-Model,SIL,NA,"{""cic_days_since_last_inquiry"": 349.0, ""cic_cn...",Train,2024-09-16 19:00:58,NaT,2024-09
1,1224007,9d9e22a5-a7d0-40fb-9360-019990c92cc7,NaN,2025-10-30T17:25:07.643489,2025-10-30T17:25:07.643489,Alpha - CIC-SIL-Model,v1,SIL-Instore,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""cic_zero_non_granted_ever_flag"": 1, ""cic_zer...",Train,2024-09-06 10:54:33,NaT,2024-09
2,1266450,a87378d6-1461-42a4-a47b-8815092f07f9,0.128524,2025-10-30T17:24:48.874131,2025-10-30T17:24:48.874131,Alpha - CIC-SIL-Model,v1,SIL-Instore,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""cic_Personal_Loans_granted_contracts_amt_24M...",Train,2024-03-01 10:51:18,NaT,2024-03
3,1269886,c134c037-58c3-4956-be66-ac1b602f0b71,NaN,2025-10-30T17:25:03.829487,2025-10-30T17:25:03.829487,Alpha - CIC-SIL-Model,v1,SIL ZERO,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""cic_zero_non_granted_ever_flag"": 1, ""cic_zer...",Train,2024-10-07 13:20:48,NaT,2024-10
4,1356848,856ad38d-2ddf-46b6-8a60-089649045a14,0.121799,2025-10-30T17:24:48.709544,2025-10-30T17:24:48.709544,Alpha - CIC-SIL-Model,v1,SIL Competitor,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""cic_days_since_last_inquiry"": 2232.0, ""cic_z...",Train,2024-11-18 19:54:43,2024-11-18 19:59:31,2024-11


In [31]:
# Expand the calcFeatures column
expanded_df1 = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df1.shape[1]}")
expanded_df1.head()

Original columns: 19
Expanded columns: 28


,customerId,digitalLoanAccountId,Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month,calc_cic_days_since_last_inquiry,calc_cic_cnt_active_contracts,calc_cic_max_amt_granted_24M,calc_cic_zero_non_granted_ever_flag,calc_cic_tot_active_contracts_util,calc_cic_vel_contract_granted_amt_12on24,calc_cic_zero_granted_ever_flag,calc_cic_Personal_Loans_granted_contracts_amt_24M,calc_cic_vel_contract_nongranted_cnt_12on24
0,1125881,aac8ee2c-dfcc-4b20-bece-61a48deb1723,0.109893,2025-10-30T17:24:52.984291,2025-10-30T17:24:52.984291,Alpha - CIC-SIL-Model,v1,SIL-Instore,F,Appliance,ios,Alpha - CIC-SIL-Model,SIL,NA,"{""cic_days_since_last_inquiry"": 349.0, ""cic_cn...",Train,2024-09-16 19:00:58,NaT,2024-09,349.0,2.0,30000.0,0,0.9418,1.0,0,NaN,NaN
1,1224007,9d9e22a5-a7d0-40fb-9360-019990c92cc7,NaN,2025-10-30T17:25:07.643489,2025-10-30T17:25:07.643489,Alpha - CIC-SIL-Model,v1,SIL-Instore,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""cic_zero_non_granted_ever_flag"": 1, ""cic_zer...",Train,2024-09-06 10:54:33,NaT,2024-09,NaN,NaN,NaN,1,NaN,NaN,1,NaN,NaN
2,1266450,a87378d6-1461-42a4-a47b-8815092f07f9,0.128524,2025-10-30T17:24:48.874131,2025-10-30T17:24:48.874131,Alpha - CIC-SIL-Model,v1,SIL-Instore,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""cic_Personal_Loans_granted_contracts_amt_24M...",Train,2024-03-01 10:51:18,NaT,2024-03,322.0,NaN,3000.0,1,NaN,0.5,0,2000.0,NaN
3,1269886,c134c037-58c3-4956-be66-ac1b602f0b71,NaN,2025-10-30T17:25:03.829487,2025-10-30T17:25:03.829487,Alpha - CIC-SIL-Model,v1,SIL ZERO,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""cic_zero_non_granted_ever_flag"": 1, ""cic_zer...",Train,2024-10-07 13:20:48,NaT,2024-10,NaN,NaN,NaN,1,NaN,NaN,1,NaN,NaN
4,1356848,856ad38d-2ddf-46b6-8a60-089649045a14,0.121799,2025-10-30T17:24:48.709544,2025-10-30T17:24:48.709544,Alpha - CIC-SIL-Model,v1,SIL Competitor,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""cic_days_since_last_inquiry"": 2232.0, ""cic_z...",Train,2024-11-18 19:54:43,2024-11-18 19:59:31,2024-11,2232.0,NaN,NaN,1,NaN,NaN,0,NaN,NaN


In [26]:
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349396 entries, 0 to 349395
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   customerId            349396 non-null  Int64  
 1   digitalLoanAccountId  349396 non-null  object 
 2   prediction            204626 non-null  float64
 3   start_time            349396 non-null  object 
 4   end_time              349396 non-null  object 
 5   modelDisplayName      349396 non-null  object 
 6   modelVersionId        349396 non-null  object 
 7   calcFeature           349396 non-null  object 
dtypes: Int64(1), float64(1), object(6)
memory usage: 21.7+ MB


In [ ]:
sq = """
with base as
(select
r.customerId,
r.digitalLoanAccountId,
r.cic_score prediction,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) start_time,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) end_time,
'Alpha-Cash-CIC-Model' modelDisplayName,
'v1' modelVersionId,
loanmaster.new_loan_type,
loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
  when lower(loanmaster.deviceType) like '%andro%' then 'android'
  else 'ios' end osType,
'SIL' as product,
r.cic_score Alpha_cic_sil_score,
r.cic_called_flag,
r.cic_hit_flag,
r.cic_Personal_Loans_granted_contracts_amt_24M,
r.cic_days_since_last_inquiry,
r.cic_cnt_active_contracts,
r.cic_vel_contract_nongranted_cnt_12on24,
r.cic_max_amt_granted_24M,
r.cic_zero_non_granted_ever_flag,
r.cic_tot_active_contracts_util,
r.cic_vel_contract_granted_amt_12on24,
r.cic_zero_granted_ever_flag,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
loanmaster.disbursementDateTime,
format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
where cic_score is not null
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) desc) = 1
)
select
  base.customerId,
  base.digitalLoanAccountId,
  base.Alpha_cic_sil_score,
  base.start_time,
  base.end_time,
  base.modelDisplayName,
  base.modelVersionId,
  base.new_loan_type,
  base.gender,
  base.loan_product_type,
  base.osType,
  'Alpha - CIC-SIL-Model' Model_Name,
  base.product,
  'NA' trenchCategory,
  ''calcFeatures,
  appln_submit_datetime,
  disbursementDateTime,
  Application_month,
  ' ' calc_run_date,
  'Train' Data_selection,
  base.cic_Personal_Loans_granted_contracts_amt_24M calc_cic_Personal_Loans_granted_contracts_amt_24M,
  base.cic_days_since_last_inquiry calc_cic_days_since_last_inquiry,
  base.cic_cnt_active_contracts calc_cic_cnt_active_contracts,
  base.cic_vel_contract_nongranted_cnt_12on24 calc_cic_vel_contract_nongranted_cnt_12on24,
  base.cic_max_amt_granted_24M calc_cic_max_amt_granted_24M,
  base.cic_zero_non_granted_ever_flag calc_cic_zero_non_granted_ever_flag,
  base.cic_tot_active_contracts_util calc_cic_tot_active_contracts_util,
  base.cic_vel_contract_granted_amt_12on24 calc_cic_vel_contract_granted_amt_12on24,
  base.cic_zero_granted_ever_flag calc_cic_zero_granted_ever_flag
from base
Where date(appln_submit_datetime) >= '2024-06-01'
;
"""
df1 = client.query(sq).to_dataframe(progress_bar_type="tqdm")
print(f"The shape of the dataframe is: {df1.shape}")
df1.head()



In [ ]:
df1.groupby('Application_month').size()

##### concatenation

In [ ]:
df_concat = pd.concat([expanded_df, df1], ignore_index=True)
df_concat.head()

In [ ]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

In [ ]:
df_concat.groupby(['Data_selection','Application_month'])['digitalLoanAccountId'].nunique().reset_index().sort_values(by=['Application_month','Data_selection'])

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()

In [ ]:
df['Alpha_cic_sil_score'] = pd.to_numeric(df['Alpha_cic_sil_score'], errors='coerce')

##### PSI calculation

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['Alpha_cic_sil_score',  'calc_cic_Personal_Loans_granted_contracts_amt_24M',
       'calc_cic_days_since_last_inquiry', 'calc_cic_cnt_active_contracts',
       'calc_cic_vel_contract_nongranted_cnt_12on24',
       'calc_cic_max_amt_granted_24M', 'calc_cic_zero_non_granted_ever_flag',
       'calc_cic_tot_active_contracts_util',
       'calc_cic_vel_contract_granted_amt_12on24',
       'calc_cic_zero_granted_ever_flag']

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                                   'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
                                    'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                                    'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cic_sil_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cic_sil_model_csi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

#### Alpha - IncomeEstimationModel

##### Query from ml_model_run_details

In [ ]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName = 'Alpha  - IncomeEstimationModel')
SELECT
  r.customerId,r.digitalLoanAccountId,prediction Alpha_Income_Estimated_score,start_time,end_time,modelDisplayName,modelVersionId,
  loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
  'Alpha  - IncomeEstimationModel' Model_Name,
  'SIL' as product,
  'NA' trenchCategory,
  r.calcFeature calcFeatures,
  'Test' Data_selection,
  JSON_VALUE(calcFeature, "$.inc_alpha_cic_credit_avg_credit_limit") AS inc_alpha_cic_credit_avg_credit_limit,
  JSON_VALUE(calcFeature, "$.inc_alpha_cic_max_active_contracts_amt") AS inc_alpha_cic_max_active_contracts_amt,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_age") AS inc_alpha_ln_age,
  JSON_VALUE(calcFeature, "$.inc_alpha_doc_type_rolled") AS inc_alpha_doc_type_rolled,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_brand") AS inc_alpha_ln_brand,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_city") AS inc_alpha_ln_city,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_cnt_dependents") AS inc_alpha_ln_cnt_dependents,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_education_level") AS inc_alpha_ln_education_level,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_employment_type_new") AS inc_alpha_ln_employment_type_new,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_gender") AS inc_alpha_ln_gender,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_industry_new") AS inc_alpha_ln_industry_new,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_loan_prod_type") AS inc_alpha_ln_loan_prod_type,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_marital_status_new") AS inc_alpha_ln_marital_status_new,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_nature_of_work_new") AS inc_alpha_ln_nature_of_work_new,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_osversion_bin") AS inc_alpha_ln_osversion_bin,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_purpose") AS inc_alpha_ln_purpose,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_source_of_funds_new") AS inc_alpha_ln_source_of_funds_new,
  JSON_VALUE(calcFeature, "$.inc_alpha_encoded_company_name_grouped") AS inc_alpha_encoded_company_name_grouped,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd.head()

###### expanded_df

In [ ]:
# Expand the calcFeatures column
expanded_df = dfd.copy()

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


##### Query from risk_mart.sil_risk_ds_master_20230101_20250309_v2

In [ ]:
sq = """
with base as
(select r.customerId,
r.digitalLoanAccountId,
r.alpha_estimated_income prediction,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) start_time,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) end_time,
'Alpha  - IncomeEstimationModel' modelDisplayName,
'v1' modelVersionId,
loanmaster.new_loan_type,
loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
  when lower(loanmaster.deviceType) like '%andro%' then 'android'
  else 'ios' end osType,
'SIL' as product,
r.alpha_estimated_income,
r.inc_alpha_cic_credit_avg_credit_limit,
r.inc_alpha_cic_max_active_contracts_amt,
r.inc_alpha_ln_company_name,
r.inc_alpha_ln_age,
r.inc_alpha_doc_type_rolled,
r.inc_alpha_ln_brand,
r.inc_alpha_ln_city,
r.inc_alpha_ln_cnt_dependents,
r.inc_alpha_ln_education_level,
r.inc_alpha_ln_employment_type_new,
r.inc_alpha_ln_gender,
r.inc_alpha_ln_industry_new,
r.inc_alpha_ln_loan_prod_type,
r.inc_alpha_ln_marital_status_new,
r.inc_alpha_ln_nature_of_work_new,
r.inc_alpha_ln_osversion_bin,
r.inc_alpha_ln_purpose,
r.inc_alpha_ln_source_of_funds_new,
r.inc_alpha_loan_monthly_income,
r.inc_alpha_encoded_company_name_grouped,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
loanmaster.disbursementDateTime,
format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
where r.alpha_estimated_income is not null
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) desc) = 1
)
select
  base.customerId,
  base.digitalLoanAccountId,
  base.alpha_estimated_income Alpha_Income_Estimated_score,
  base.start_time,
  base.end_time,
  base.modelDisplayName,
  base.modelVersionId,
  base.new_loan_type,
  base.gender,
  base.loan_product_type,
  base.osType,
  'Alpha  - IncomeEstimationModel' Model_Name,
  base.product,
  'NA' trenchCategory,
  ''calcFeatures,
  ' ' calc_run_date,
  'Train' Data_selection,
  base.inc_alpha_cic_credit_avg_credit_limit,
  base.inc_alpha_cic_max_active_contracts_amt,
  base.inc_alpha_ln_age,
  base.inc_alpha_doc_type_rolled,
  base.inc_alpha_ln_brand,
  base.inc_alpha_ln_city,
  base.inc_alpha_ln_cnt_dependents,
  base.inc_alpha_ln_education_level,
  base.inc_alpha_ln_employment_type_new,
  base.inc_alpha_ln_gender,
  base.inc_alpha_ln_industry_new,
  base.inc_alpha_ln_loan_prod_type,
  base.inc_alpha_ln_marital_status_new,
  base.inc_alpha_ln_nature_of_work_new,
  base.inc_alpha_ln_osversion_bin,
  base.inc_alpha_ln_purpose,
  base.inc_alpha_ln_source_of_funds_new,
  base.inc_alpha_encoded_company_name_grouped,
  base.appln_submit_datetime,
  base.disbursementDateTime,
  base.Application_month
from base
Where date(appln_submit_datetime) >= '2024-06-01'
;

"""
df1 = client.query(sq).to_dataframe(progress_bar_type="tqdm")
print(f"The shape of the dataframe is: {df1.shape}")
df1.head()



##### concatenation

In [ ]:
df_concat = pd.concat([expanded_df, df1], ignore_index=True)
df_concat.head()

In [ ]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

In [ ]:
df_concat.groupby(['Data_selection','Application_month'])['digitalLoanAccountId'].nunique().reset_index().sort_values(by=['Application_month','Data_selection'])

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()

In [ ]:
df['Alpha_Income_Estimated_score'] = pd.to_numeric(df['Alpha_Income_Estimated_score'], errors='coerce')

In [ ]:
df['inc_alpha_cic_credit_avg_credit_limit'] = pd.to_numeric(df['inc_alpha_cic_credit_avg_credit_limit'], errors ='coerce')
df['inc_alpha_cic_max_active_contracts_amt'] = pd.to_numeric(df['inc_alpha_cic_max_active_contracts_amt'], errors='coerce')
df['inc_alpha_encoded_company_name_grouped'] = pd.to_numeric(df['inc_alpha_encoded_company_name_grouped'], errors='coerce')
df.info()


##### PSI calculation

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['Alpha_Income_Estimated_score',   'inc_alpha_cic_credit_avg_credit_limit',
       'inc_alpha_cic_max_active_contracts_amt', 'inc_alpha_ln_age',
       'inc_alpha_doc_type_rolled', 'inc_alpha_ln_brand', 'inc_alpha_ln_city',
       'inc_alpha_ln_cnt_dependents', 'inc_alpha_ln_education_level',
       'inc_alpha_ln_employment_type_new', 'inc_alpha_ln_gender',
       'inc_alpha_ln_industry_new', 'inc_alpha_ln_loan_prod_type',
       'inc_alpha_ln_marital_status_new', 'inc_alpha_ln_nature_of_work_new',
       'inc_alpha_ln_osversion_bin', 'inc_alpha_ln_purpose',
       'inc_alpha_ln_source_of_funds_new',
       'inc_alpha_encoded_company_name_grouped',]

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                                   'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
                                    'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                                    'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_income_estimation_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_income_estimation_model_csi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

#### Alpha Sil Stack Model

##### Query from ml_model_run_details

In [ ]:
sq = """ WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName = 'Alpha - StackingModel')
SELECT distinct
 r.customerId,r.digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'SIL Alpha - StackingModel' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature calcFeatures,
  'Test' Data_selection,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over(partition by r.digitalLoanAccountId order by r.start_time desc)=1
  ;"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

###### expanded_df

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


##### Query from risk_mart.sil_risk_ds_master_20230101_20250309_v2

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
select *  from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` limit 1
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

In [ ]:
sq = """
with base as
(select
r.customerId,
r.digitalLoanAccountId,
r.alpha_stack_score prediction,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) start_time,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) end_time,
'Alpha - StackingModel' modelDisplayName,
'v1' modelVersionId,
loanmaster.new_loan_type,
loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
  when lower(loanmaster.deviceType) like '%andro%' then 'android'
  else 'ios' end osType,
'SIL' as product,
r.alpha_stack_score Sil_Alpha_Stack_score,
r.beta_demo_score,
r.cic_score,
r.apps_score,
r.credo_gen_score,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
loanmaster.disbursementDateTime,
format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
where alpha_stack_score is not null
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) desc) = 1
)
select
  base.customerId,
  base.digitalLoanAccountId,
  base.Sil_Alpha_Stack_score,
  base.start_time,
  base.end_time,
  base.modelDisplayName,
  base.modelVersionId,
  base.new_loan_type,
  base.gender,
  base.loan_product_type,
  base.osType,
  'Alpha - StackingModel' Model_Name,
  base.product,
  'NA' trenchCategory,
  ''calcFeatures,
  appln_submit_datetime,
  disbursementDateTime,
  Application_month,
  'Train' Data_selection,
  base.beta_demo_score calc_sb_demo_score,
  base.cic_score calc_s_cic_score,
  base.credo_gen_score calc_s_credo_score,
  base.apps_score calc_s_apps_score
from base
Where date(appln_submit_datetime) >= '2024-06-01'
;
"""
df1 = client.query(sq).to_dataframe(progress_bar_type="tqdm")
print(f"The shape of the dataframe is: {df1.shape}")
df1.head()



##### concatenation

In [ ]:
df_concat = pd.concat([expanded_df, df1], ignore_index=True)
df_concat.head()

In [ ]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()

In [ ]:
df.info()

In [ ]:
df['Sil_Alpha_Stack_score'] = pd.to_numeric(df['Sil_Alpha_Stack_score'], errors='coerce')

##### PSI calculation

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['Sil_Alpha_Stack_score',   'calc_sb_demo_score',
       'calc_s_cic_score', 'calc_s_credo_score', 'calc_s_apps_score']

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                                   'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
                                    'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                                    'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_sil_stack_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_sil_stack_model_csi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

#### Beta Sil App Score

##### ml_model_run_details

In [ ]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName = 'Beta - AppsScoreModel')
SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'SIL Beta - AppsScoreModel' Model_Name,
 'SIL' as product,
  'NA' trenchCategory,
  'Test' Data_selection,
  safe_cast(JSON_VALUE(prediction_clean, "$.combined_score") AS float64) as sil_beta_app_score,
 calcFeature calcFeatures,
    coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
    loanmaster.disbursementDateTime,
    format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
 FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by start_time desc) = 1
;
"""
dfd = client.query(sq).to_dataframe()
dfd.head()

##### expanded_df

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
select * from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r limit 1
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

##### Query from risk_mart_sil_risk_ds_master_20230101_20250309_v2

In [ ]:
sq = """
with base as
(select
r.customerId,
r.digitalLoanAccountId,
r.alpha_stack_score prediction,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) start_time,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) end_time,
'Beta - AppsScoreModel' modelDisplayName,
'v1' modelVersionId,
loanmaster.new_loan_type,
loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
  when lower(loanmaster.deviceType) like '%andro%' then 'android'
  else 'ios' end osType,
'SIL' as product,
r.apps_score sil_beta_app_score,
r.app_cnt_absence_tag_30d calc_app_cnt_absence_tag_30d,
r.app_cnt_absence_tag_90d calc_app_cnt_absence_tag_90d,
r.app_cnt_business_ever calc_app_cnt_business_ever,
r.app_cnt_competitors_30d calc_app_cnt_competitors_30d,
r.app_cnt_competitors_90d calc_app_cnt_competitors_90d,
r.app_cnt_education_ever calc_app_cnt_education_ever,
r.app_cnt_finance_7d calc_app_cnt_finance_7d,
r.app_cnt_finance_90d calc_app_cnt_finance_90d,
r.app_cnt_music_and_audio_ever calc_app_cnt_music_and_audio_ever,
r.app_cnt_payday_90d calc_app_cnt_payday_90d,
r.app_cnt_rated_for_3plus_ever calc_app_cnt_rated_for_3plus_ever,
r.app_cnt_travel_and_local_ever calc_app_cnt_travel_and_local_ever,
r.app_first_competitors_install_to_apply_days calc_app_first_competitors_install_to_apply_days,
r.app_first_payday_install_to_apply_days calc_app_first_payday_install_to_apply_days,
r.app_median_time_bw_installed_mins_30d calc_app_median_time_bw_installed_mins_30d,
r.app_vel_finance_30_over_365 calc_app_vel_finance_30_over_365,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
loanmaster.disbursementDateTime,
format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
where apps_score is not null
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) desc) = 1
)
select
  base.customerId,
  base.digitalLoanAccountId,
  base.sil_beta_app_score,
  base.start_time,
  base.end_time,
  base.modelDisplayName,
  base.modelVersionId,
  base.new_loan_type,
  base.gender,
  base.loan_product_type,
  base.osType,
  'Beta - AppsScoreModel' Model_Name,
  base.product,
  'NA' trenchCategory,
  ''calcFeatures,
  appln_submit_datetime,
  disbursementDateTime,
  Application_month,
  'Train' Data_selection,
  base.calc_app_cnt_rated_for_3plus_ever,
  base.calc_app_cnt_education_ever,
  base.calc_app_cnt_business_ever,
  base.calc_app_cnt_music_and_audio_ever,
  base.calc_app_cnt_travel_and_local_ever,
  base.calc_app_cnt_finance_7d,
  base.calc_app_cnt_competitors_30d,
  base.calc_app_cnt_absence_tag_30d,
  -- base.calc_app_cnt_finance_30d,
  base.calc_app_cnt_absence_tag_90d,
  base.calc_app_cnt_finance_90d,
  base.calc_app_cnt_competitors_90d,
  base.calc_app_cnt_payday_90d,
  --- base.calc_app_avg_time_bw_installed_mins_30d,
  base.calc_app_median_time_bw_installed_mins_30d,
  base.calc_app_first_competitors_install_to_apply_days,
  base.calc_app_first_payday_install_to_apply_days,
  base.calc_app_vel_finance_30_over_365
from base
Where date(appln_submit_datetime) >= '2024-06-01'
;
"""
df1 = client.query(sq).to_dataframe(progress_bar_type="tqdm")
print(f"The shape of the dataframe is: {df1.shape}")
df1.head()



In [ ]:
df1.columns

In [ ]:
expanded_df = expanded_df[['customerId', 'digitalLoanAccountId', 'prediction', 'start_time',
       'end_time', 'modelDisplayName', 'modelVersionId', 'new_loan_type',
       'gender', 'loan_product_type', 'osType', 'Model_Name', 'product',
       'trenchCategory', 'Data_selection', 'sil_beta_app_score',
       'calcFeatures', 'appln_submit_datetime', 'disbursementDateTime',
       'Application_month', 'calc_app_cnt_rated_for_3plus_ever',
       'calc_app_cnt_education_ever', 'calc_app_cnt_business_ever',
       'calc_app_cnt_music_and_audio_ever',
       'calc_app_cnt_travel_and_local_ever', 'calc_app_cnt_finance_7d',
       'calc_app_cnt_competitors_30d', 'calc_app_cnt_absence_tag_30d',
        'calc_app_cnt_absence_tag_90d',
       'calc_app_cnt_finance_90d', 'calc_app_cnt_competitors_90d',
       'calc_app_cnt_payday_90d',
       'calc_app_median_time_bw_installed_mins_30d',
       'calc_app_first_competitors_install_to_apply_days',
       'calc_app_first_payday_install_to_apply_days',
       'calc_app_vel_finance_30_over_365']].copy()

##### concatenation

In [ ]:
df_concat = pd.concat([expanded_df, df1], ignore_index=True)
df_concat.info()

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()

In [ ]:

df['sil_beta_app_score'] = pd.to_numeric(df['sil_beta_app_score'], errors='coerce')

##### PSI calculation

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['sil_beta_app_score',
       'calc_app_cnt_rated_for_3plus_ever', 'calc_app_cnt_education_ever',
       'calc_app_cnt_business_ever', 'calc_app_cnt_music_and_audio_ever',
       'calc_app_cnt_travel_and_local_ever', 'calc_app_cnt_finance_7d',
       'calc_app_cnt_absence_tag_30d', 'calc_app_cnt_competitors_30d',
       'calc_app_cnt_absence_tag_90d',    'calc_app_cnt_finance_90d', 'calc_app_cnt_competitors_90d',
       'calc_app_cnt_payday_90d',      'calc_app_median_time_bw_installed_mins_30d',
       'calc_app_first_competitors_install_to_apply_days',
       'calc_app_first_payday_install_to_apply_days',
       'calc_app_vel_finance_30_over_365']

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                                   'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
                                    'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                                    'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_appscore_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_appscore_model_csi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

#### Beta SIL Demo Score

##### ml_model_run_details

In [ ]:
sq = """ WITH cleaned AS (
  SELECT
  customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName = 'Beta - DemoScoreModel')
SELECT
  r.customerId,  r.digitalLoanAccountId,start_time, prediction sil_beta_demo_score, modelDisplayName,modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
      case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
  case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
 'SIL Beta - DemoScoreModel' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature_cleaned calcFeatures,
  'Test' Data_selection,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code

qualify row_number() over(partition by r.customerId, r.digitalLoanAccountId order by start_time desc) = 1;"""

dfd = client.query(sq).to_dataframe()
dfd.head()

##### expanded_df

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


##### Query from risk_mart_sil_risk_ds_master_20230101_20250309_v2

In [ ]:
expanded_df.columns

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
select * from risk_mart.sil_risk_ds_master_20230101_20250309_v2 limit 1
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

In [ ]:
sq = """
with base as
(select
r.customerId,
r.digitalLoanAccountId,
r.beta_demo_score prediction,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) start_time,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) end_time,
'Beta - DemoScoreModel' modelDisplayName,
'v1' modelVersionId,
loanmaster.new_loan_type,
loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
  when lower(loanmaster.deviceType) like '%andro%' then 'android'
  else 'ios' end osType,
'SIL' as product,
r.beta_demo_score sil_beta_demo_score,
r.beta_de_ln_vas_opted_flag calc_beta_de_ln_vas_opted_flag,
r.beta_de_ln_doc_type_rolled calc_beta_de_ln_doc_type_rolled,
r.beta_de_ln_marital_status calc_beta_de_ln_marital_status,
r.beta_de_ln_age_bin calc_beta_de_ln_age_bin,
r.beta_de_ln_province_bin calc_beta_de_ln_province_bin,
r.beta_de_ln_ref2_type calc_beta_de_ln_ref2_type,
r.beta_de_ln_education_level calc_beta_de_ln_education_level,
r.beta_de_ln_ref1_type calc_beta_de_ln_ref1_type,
r.beta_de_ln_industry_new_bin calc_beta_de_ln_industry_new_bin,
r.beta_de_ln_appln_day_of_week calc_beta_de_ln_appln_day_of_week,
r.beta_de_onb_name_email_match_score calc_beta_de_onb_name_email_match_score,
r.beta_de_ln_employment_type_new_bin calc_beta_de_ln_employment_type_new_bin,
r.beta_de_ln_telconame calc_beta_de_ln_telconame,
r.beta_de_time_bw_onb_loan_appln_mins calc_beta_de_time_bw_onb_loan_appln_mins,
r.beta_de_ln_source_of_funds_new_bin calc_beta_de_ln_source_of_funds_new_bin,
r.beta_de_ln_brand_bin calc_beta_de_ln_brand_bin,
r.beta_de_ln_email_primary_domain calc_beta_de_ln_email_primary_domain,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
loanmaster.disbursementDateTime,
format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
where beta_demo_score is not null
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) desc) = 1
)
select
  base.customerId,
  base.digitalLoanAccountId,
  base.sil_beta_demo_score,
  base.start_time,
  base.end_time,
  base.modelDisplayName,
  base.modelVersionId,
  base.new_loan_type,
  base.gender,
  base.loan_product_type,
  base.osType,
  'Beta - DemoScoreModel' Model_Name,
  base.product,
  'NA' trenchCategory,
  ''calcFeatures,
  appln_submit_datetime,
  disbursementDateTime,
  Application_month,
  'Train' Data_selection,
  base.calc_beta_de_ln_vas_opted_flag,
  base.calc_beta_de_ln_doc_type_rolled,
  base.calc_beta_de_ln_marital_status,
  base.calc_beta_de_ln_age_bin,
  base.calc_beta_de_ln_province_bin,
  base.calc_beta_de_ln_ref2_type,
  base.calc_beta_de_ln_education_level,
  base.calc_beta_de_ln_ref1_type,
  base.calc_beta_de_ln_industry_new_bin,
  base.calc_beta_de_ln_appln_day_of_week,
  base.calc_beta_de_onb_name_email_match_score,
  base.calc_beta_de_ln_employment_type_new_bin,
  base.calc_beta_de_ln_telconame,
  base.calc_beta_de_time_bw_onb_loan_appln_mins,
  base.calc_beta_de_ln_source_of_funds_new_bin,
  base.calc_beta_de_ln_brand_bin,
  base.calc_beta_de_ln_email_primary_domain
from base
Where date(appln_submit_datetime) >= '2024-06-01'
;
"""
df1 = client.query(sq).to_dataframe(progress_bar_type="tqdm")
print(f"The shape of the dataframe is: {df1.shape}")
df1.head()



##### concatenation

In [ ]:
df_concat = pd.concat([expanded_df, df1], ignore_index=True)
df_concat.info()

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()

In [ ]:
df['sil_beta_demo_score'] = pd.to_numeric(df['sil_beta_demo_score'], errors='coerce')

##### PSI calculation

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['sil_beta_demo_score',
       'calc_beta_de_ln_vas_opted_flag',
       'calc_beta_de_ln_doc_type_rolled', 'calc_beta_de_ln_marital_status',
       'calc_beta_de_ln_age_bin', 'calc_beta_de_ln_province_bin',
       'calc_beta_de_ln_ref2_type', 'calc_beta_de_ln_education_level',
       'calc_beta_de_ln_ref1_type', 'calc_beta_de_ln_industry_new_bin',
       'calc_beta_de_ln_appln_day_of_week',
       'calc_beta_de_onb_name_email_match_score',
       'calc_beta_de_ln_employment_type_new_bin', 'calc_beta_de_ln_telconame',
       'calc_beta_de_time_bw_onb_loan_appln_mins',
       'calc_beta_de_ln_source_of_funds_new_bin', 'calc_beta_de_ln_brand_bin',
       'calc_beta_de_ln_email_primary_domain']

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                                   'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
                                    'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                                    'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_demo_score_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_demo_score_model_csi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

#### Beta SIL STACK Score Model

###### ml_model_run_details

In [ ]:
sq = """ WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName = 'Beta - StackScoreModel')
SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
 'SIL_Beta - StackScoreModel' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature calcFeatures,
  'Test' Data_selection,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId, r.digitalLoanAccountId order by start_time desc) = 1
;
"""
dfd= client.query(sq).to_dataframe()

In [ ]:
dfd.head()

##### expanded_df

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


In [ ]:
expanded_df.rename(columns = {'prediction':'sil_beta_stack_score'}, inplace = True)
expanded_df.columns

##### Query from risk_mart_sil_ds_master_20230101_20250309_v2

In [ ]:
sq = """
with base as
(select
r.customerId,
r.digitalLoanAccountId,
r.beta_stack_score prediction,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) start_time,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) end_time,
'Beta - StackScoreModel' modelDisplayName,
'v1' modelVersionId,
loanmaster.new_loan_type,
loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
  when lower(loanmaster.deviceType) like '%andro%' then 'android'
  else 'ios' end osType,
'SIL' as product,
r.beta_stack_score sil_beta_stack_score,
r.apps_score calc_s_apps_score,
r.credo_gen_score calc_s_credo_score,
r.beta_demo_score beta_demo_score,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
loanmaster.disbursementDateTime,
format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
where beta_stack_score is not null
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) desc) = 1
)
select
  base.customerId,
  base.digitalLoanAccountId,
  base.sil_beta_stack_score,
  base.start_time,
  base.end_time,
  base.modelDisplayName,
  base.modelVersionId,
  base.new_loan_type,
  base.gender,
  base.loan_product_type,
  base.osType,
  'Beta - StackScoreModel' Model_Name,
  base.product,
  'NA' trenchCategory,
  ''calcFeatures,
  appln_submit_datetime,
  disbursementDateTime,
  Application_month,
  'Train' Data_selection,
  base.calc_s_apps_score,
  base.calc_s_credo_score,
  base.beta_demo_score calc_sb_demo_score
from base
Where date(appln_submit_datetime) >= '2024-06-01'
;"""

print(sq)
df1 = client.query(sq).to_dataframe(progress_bar_type="tqdm")
print(f"The shape of the dataframe is: {df1.shape}")
df1.head()

###### Concatenation

In [ ]:
df_concat = pd.concat([expanded_df, df1], ignore_index=True)
df_concat.info()

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()

In [ ]:
df['sil_beta_stack_score'] = pd.to_numeric(df['sil_beta_stack_score'], errors='coerce')

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['sil_beta_stack_score',
        'calc_s_apps_score', 'calc_s_credo_score', 'calc_sb_demo_score']

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                                   'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
                                    'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                                    'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_stack_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_stack_model_csi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

#### Beta - IncomeEstimationModel

##### ml_model_run_details

In [ ]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName = 'Beta - IncomeEstimationModel')
SELECT
  r.customerId,r.digitalLoanAccountId,prediction sil_beta_income_estimation_score,start_time,end_time,modelDisplayName,modelVersionId,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_loan_type") AS inc_beta_ln_loan_type,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_education_level") AS inc_beta_ln_education_level,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_employment_type_new") AS inc_beta_ln_employment_type_new,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_industry_new") AS inc_beta_ln_industry_new,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_age") AS inc_beta_ln_age,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_brand") AS inc_beta_ln_brand,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_city") AS inc_beta_ln_city,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_purpose") AS inc_beta_ln_purpose,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_osversion_bin") AS inc_beta_ln_osversion_bin,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_postal_code") AS inc_beta_ln_postal_code,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_gender") AS inc_beta_ln_gender,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_doc_type_rolled") AS inc_beta_ln_doc_type_rolled,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_cnt_dependents") AS inc_beta_ln_cnt_dependents,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_source_of_funds_new") AS inc_beta_ln_source_of_funds_new,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_marital_status_new") AS inc_beta_ln_marital_status_new,
  JSON_VALUE(calcFeature, "$.inc_beta_encoded_company_name_grouped") AS inc_beta_encoded_company_name_grouped,
   loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
 'SIL Beta - IncomeEstimationModel' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature calcFeatures,
  'Test' Data_selection,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId, r.digitalLoanAccountId order by start_time desc) = 1
  ;
"""
dfd = client.query(sq).to_dataframe()


In [ ]:
dfd.head()

##### expanded_df

In [ ]:
# Expand the calcFeatures column
expanded_df = dfd.copy()

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")



##### Query from risk_mart_sil_ds_master_20230101_20250309_v2

In [ ]:
# sql_engine: bigquery
# output_variable: df
# start _sql
_sql = """
select * from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` limit 1
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
df = _bqsqlcell.run(_sql)
df

In [ ]:
sq = """
with base as
(select r.customerId,
r.digitalLoanAccountId,
r.beta_estimated_income prediction,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) start_time,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) end_time,
'Alpha  - IncomeEstimationModel' modelDisplayName,
'v1' modelVersionId,
loanmaster.new_loan_type,
loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
  when lower(loanmaster.deviceType) like '%andro%' then 'android'
  else 'ios' end osType,
'SIL' as product,
r.beta_estimated_income,
r.inc_beta_ln_company_name,
r.inc_beta_ln_loan_type,
r.inc_beta_ln_education_level,
r.inc_beta_ln_employment_type_new,
r.inc_beta_ln_industry_new,
r.inc_beta_ln_age,
r.inc_beta_ln_brand,
r.inc_beta_ln_city,
r.inc_beta_ln_purpose,
r.inc_beta_ln_osversion_bin,
r.inc_beta_ln_postal_code,
r.inc_beta_ln_gender,
r.inc_beta_ln_doc_type_rolled,
r.inc_beta_ln_cnt_dependents,
r.inc_beta_ln_source_of_funds_new,
r.inc_beta_ln_marital_status_new,
r.inc_beta_loan_monthly_income,
r.inc_beta_encoded_company_name_grouped,
IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
loanmaster.disbursementDateTime,
format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
where r.alpha_estimated_income is not null
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) desc) = 1
)
select
  base.customerId,
  base.digitalLoanAccountId,
  base.beta_estimated_income sil_beta_income_estimation_score,
  base.start_time,
  base.end_time,
  base.modelDisplayName,
  base.modelVersionId,
  base.new_loan_type,
  base.gender,
  base.loan_product_type,
  base.osType,
  'Alpha  - IncomeEstimationModel' Model_Name,
  base.product,
  'NA' trenchCategory,
  ''calcFeatures,
  ' ' calc_run_date,
  'Train' Data_selection,
  base.inc_beta_ln_loan_type,
  base.inc_beta_ln_education_level,
  base.inc_beta_ln_employment_type_new,
  base.inc_beta_ln_industry_new,
  base.inc_beta_ln_age,
  base.inc_beta_ln_brand,
  base.inc_beta_ln_city,
  base.inc_beta_ln_purpose,
  base.inc_beta_ln_osversion_bin,
  base.inc_beta_ln_postal_code,
  base.inc_beta_ln_gender,
  base.inc_beta_ln_doc_type_rolled,
  base.inc_beta_ln_cnt_dependents,
  base.inc_beta_ln_source_of_funds_new,
  base.inc_beta_ln_marital_status_new,
  base.inc_beta_encoded_company_name_grouped,
  base.appln_submit_datetime,
  base.disbursementDateTime,
  base.Application_month
from base
Where date(appln_submit_datetime) >= '2024-06-01'
;

"""
df1 = client.query(sq).to_dataframe(progress_bar_type="tqdm")
print(f"The shape of the dataframe is: {df1.shape}")
df1.head()



In [ ]:
df_concat = pd.concat([expanded_df, df1], ignore_index=True)
df_concat.info()

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()
df['sil_beta_income_estimation_score'] = pd.to_numeric(df['sil_beta_income_estimation_score'], errors='coerce')

### run psi calculation

df = df.copy()

# Define feature list
feature_list = ['sil_beta_income_estimation_score',  'inc_beta_ln_loan_type',
       'inc_beta_ln_education_level', 'inc_beta_ln_employment_type_new',
       'inc_beta_ln_industry_new', 'inc_beta_ln_age', 'inc_beta_ln_brand',
       'inc_beta_ln_city', 'inc_beta_ln_purpose', 'inc_beta_ln_osversion_bin',
       'inc_beta_ln_postal_code', 'inc_beta_ln_gender',
       'inc_beta_ln_doc_type_rolled', 'inc_beta_ln_cnt_dependents',
       'inc_beta_ln_source_of_funds_new', 'inc_beta_ln_marital_status_new',
       'inc_beta_encoded_company_name_grouped',]

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                                   'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
                                    'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                                    'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_income_estimation_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_income_estimation_model_csi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

# Graveyard

---



In [ ]:
df_concat.info()

In [ ]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cic_sil_model_csi_v4;"""
dfd = client.query(sq).to_dataframe()
dfd.head()



In [ ]:
dd.query("""select Base_Month, Current_Month, Bin , Bin_Range, Base_Percentage,	Actual_Percentage, Bin_PSI from dfd
where modelDisplayName = 'Alpha - CIC-SIL-Model' and Feature = 'calc_cic_zero_non_granted_ever_flag' and Segment_Column = 'Overall'
order by 1,2;""").to_df()

In [ ]:
dd.query("""select Application_month,
min(case when calc_sb_demo_score is not null then calc_sb_demo_score end) min_calc_sb_demo_score,
max(case when calc_sb_demo_score is not null then calc_sb_demo_score end) max_calc_sb_demo_score,
count(distinct case when calc_sb_demo_score is null then digitalLoanAccountId end) cnt_calc_sb_demo_score_missing,
min(case when calc_s_cic_score is not null then calc_s_cic_score end) min_calc_s_cic_score,
max(case when calc_s_cic_score is not null then calc_s_cic_score end) max_calc_s_cic_score,
count(distinct case when calc_s_cic_score is null then digitalLoanAccountId end) cnt_s_cic_score_missing,
min(case when calc_s_credo_score is not null then calc_s_credo_score end) min_calc_s_credo_score,
max(case when calc_s_credo_score is not null then calc_s_credo_score end) max_calc_s_credo_score,
count(distinct case when calc_s_credo_score is null then digitalLoanAccountId end) cnt_calc_s_credo_score_missing,
min(case when calc_s_apps_score is not null then calc_s_apps_score end) min_calc_s_apps_score,
max(case when calc_s_apps_score is not null then calc_s_apps_score end) max_calc_s_apps_score,
count(distinct case when calc_s_apps_score is null then digitalLoanAccountId end) cnt_calc_s_apps_score_missing,
from
expanded_df
group by 1
order by 1

"""
).to_df()